# Multi-hypothesis approach

In [1]:
import numpy as np
import random

from scipy import stats

In [2]:
# Probability of each hypothesis
# 0th entry is for the null hypothesis
p_hypothesis = [0.25, 0.25, 0.25, 0.25]
assert sum(p_hypothesis) == 1.0

In [3]:
# Parameters of the null hypothesis
lower_bound = 0
upper_bound = 100

# Mean of the Poisson distribution of each hypothesis
mu = [0.1, 2.0, 5.0]

In [4]:
# Number of experiments to perform
n_experiments = 10000

# Define the confusion matrix
n_hypotheses = len(p_hypothesis)
confusion_matrix = np.zeros((n_hypotheses, n_hypotheses))

In [5]:
def p_D_given_null(D):
    """Probability of the D given the null hypothesis."""
    
    if lower_bound <= sample <= upper_bound:
        n_samples = upper_bound - lower_bound + 1
        return 1.0 / n_samples
    else:
        return 0.0

In [6]:
def p_D_given_Poisson(D, mu):
    """p(D|mu)."""
        
    return stats.poisson.pmf(D, mu) 

In [7]:
def p_hypothesis_given_data(D):
    
    p = np.zeros(len(p_hypothesis))
    p[0] = p_D_given_null(D) * p_hypothesis[0]
    
    for i in range(1, len(p_hypothesis)):
        p[i] = p_D_given_Poisson(D, mu[i-1]) * p_hypothesis[i]
    
    return p / sum(p)

In [8]:
for i in range(n_experiments):
    
    # Index of the true hypothesis
    true_hypothesis = np.where(np.random.multinomial(1, p_hypothesis) == 1)[0][0]
    
    # Generate a sample
    if true_hypothesis == 0:
        sample = random.randint(lower_bound, upper_bound)
    else:
        mu_true = mu[true_hypothesis-1]
        sample = np.random.poisson(mu_true)
    
    # Probability of each hypothesis given the data
    p_est = p_hypothesis_given_data(sample)
    
    # Index of the most likely hypothesis given the data
    est_hypothesis = np.argmax(p_est)
    
    confusion_matrix[true_hypothesis, est_hypothesis] += 1

In [9]:
confusion_matrix

array([[2225.,   27.,   86.,  189.],
       [   0., 2271.,  217.,    0.],
       [   0.,  338., 1820.,  360.],
       [  43.,   17.,  651., 1756.]])

## Multiple lotteries

* Person enters $N$ lotteries
* Each lottery has the same probability of winning

In [31]:
n_lotteries = 2
p_win = 0.1

buy_island_model_0 = {
    True: 1.0,
    False: 0.0
}

buy_island_model_1 = {
    0: 0,
    1: 1.0,
    2: 1.0,
    3: 1.0,
}

In [32]:
def bernoulli_sample(p):
    assert 0.0 <= p <= 1.0
    return np.random.binomial(1, p)

In [33]:
def buy_island_0(win):
    assert type(win) == bool
    
    p = buy_island_model_0[win]
    return bernoulli_sample(p)

In [34]:
def buy_island_1(n_wins):
    assert type(n_wins) == int
    assert n_wins >= 0
    
    p = buy_island_model_1[n_wins]
    return bernoulli_sample(p)

In [35]:
def run_experiment(n_lotteries, p_win):
    assert n_lotteries > 0
    assert 0.0 <= p_win <= 1.0
    
    # Determine the outcome of each lottery
    outcomes = [bernoulli_sample(p_win) for i in range(n_lotteries)]
    
    number_of_wins = sum(outcomes)
    assert 0 <= number_of_wins <= n_lotteries
    
    # Model 0
    buy_0 = buy_island_0(number_of_wins > 0)
    
    # Model 1
    buy_1 = buy_island_1(number_of_wins)
    
    return buy_0, buy_1

In [40]:
confusion_matrix = np.zeros((2, 2))

n_experiments = 100000
for i in range(n_experiments):
    buy_0, buy_1 = run_experiment(n_lotteries, p_win)
    confusion_matrix[buy_0, buy_1] += 1

confusion_matrix

array([[80818.,     0.],
       [    0., 19182.]])